https://www.nature.com/articles/s41598-020-80132-4

In [ ]:
!pip install brian2 jupyter numpy scipy matplotlib brian2tools

     |████████████████████████████████| 1.2MB 9.1MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 
  Created wheel for brian2: filename=Brian2-2.4.2-cp37-cp37m-linux_x86_64.whl size=1257128 sha256=53b4fa1ca21e0e89ee37c2adad10d9701ab77473f5d6a87bb07cbe878c6050ab
  Stored in directory: /root/.cache/pip/wheels/5d/4b/cf/ef50e71df5479d67f981cbb17b1d842381019d27ba144d57de
Successfully built brian2


In [ ]:
import numpy as np
import scipy
from brian2 import *
import matplotlib.pyplot as plt
import random
from brian2tools import *
import sys
from google.colab import drive
from PIL import Image
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/552fpdata')


Mounted at /content/gdrive


In [ ]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

import os
import cv2

def gaussian_noise(image):
    row,col= image.shape
    mean = 0
    var = 0.05
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = image + gauss
    return noisy


In [ ]:
filepath = '/content/gdrive/MyDrive/552fpdata'
filenames = ['0.png', '10.png', '20.png', '30.png', '40.png', '50.png', '60.png', '70.png', '80.png', '90.png']

image_data = []
image_labels = [] 
#Make Dataset
for file in filenames:
  a = load_image(filepath + '/' + file)
  a = a[:,:,0]
  for i in range(100):
    b = gaussian_noise(a)
    image_data.append(b)
    image_labels.append(file[:len(file) - 4])

image_data = np.array(image_data) / 255
image_labels = np.array(image_labels)

In [ ]:
spike_data = []
spike_labels = []
times= []
from matplotlib.image import imread

img = (1-imread(filepath + '/' + filenames[4]))[::-1, :, 0].T
num_samples, N = img.shape
img = gaussian_noise(img)
start_scope()
ta = TimedArray(img, dt=1*ms) # 228

In [ ]:
max = img[0,0]
for i in range(len(img)):
  for k in range(len(img)):
    if (img[i,k]>max):
      max = img[i,k]
max

1.7073979194565365

In [ ]:
min = img[0,0]
for i in range(len(img)):
  for k in range(len(img)):
    if (img[i,k]<min):
      min = img[i,k]
min

-0.7117550091449683

In [ ]:
spike_data = []
spike_labels = []
times= []
from matplotlib.image import imread
for files in filenames:
  for k in range(100):
    img = (1-imread(filepath + '/' + files))[::-1, :, 0].T
    num_samples, N = img.shape
    img = gaussian_noise(img)
    start_scope()
    ta = TimedArray(img, dt=1*ms) # 228
    A = 1.5
    tau = 2*ms
    eqs = '''
    dv/dt = (A*ta(t, i)-v)/tau : 1
    '''
    G = NeuronGroup(N, eqs, threshold='v>1', reset='v=0', method='euler')

    M = SpikeMonitor(G)
    run(num_samples*ms)
    times.append(np.array(M.t))
    spike_data.append(np.array(M.i))
    spike_labels.append(file[:len(file) - 4])
    print("Angle: " + files + ". Repeat: " + str(k))

Angle: 0.png. Repeat: 0
Angle: 0.png. Repeat: 1
Angle: 0.png. Repeat: 2
Angle: 0.png. Repeat: 3
Angle: 0.png. Repeat: 4
Angle: 0.png. Repeat: 5
Angle: 0.png. Repeat: 6
Angle: 0.png. Repeat: 7
Angle: 0.png. Repeat: 8
Angle: 0.png. Repeat: 9
Angle: 0.png. Repeat: 10
Angle: 0.png. Repeat: 11
Angle: 0.png. Repeat: 12
Angle: 0.png. Repeat: 13
Angle: 0.png. Repeat: 14
Angle: 0.png. Repeat: 15
Angle: 0.png. Repeat: 16
Angle: 0.png. Repeat: 17
Angle: 0.png. Repeat: 18
Angle: 0.png. Repeat: 19
Angle: 0.png. Repeat: 20
Angle: 0.png. Repeat: 21
Angle: 0.png. Repeat: 22
Angle: 0.png. Repeat: 23
Angle: 0.png. Repeat: 24
Angle: 0.png. Repeat: 25
Angle: 0.png. Repeat: 26
Angle: 0.png. Repeat: 27
Angle: 0.png. Repeat: 28
Angle: 0.png. Repeat: 29
Angle: 0.png. Repeat: 30
Angle: 0.png. Repeat: 31
Angle: 0.png. Repeat: 32
Angle: 0.png. Repeat: 33
Angle: 0.png. Repeat: 34
Angle: 0.png. Repeat: 35
Angle: 0.png. Repeat: 36
Angle: 0.png. Repeat: 37
Angle: 0.png. Repeat: 38
Angle: 0.png. Repeat: 39
Angle: 0.p

In [ ]:
#Convert i and t to matrix
def convitToMat(i, t):
  toRet = np.zeros(shape=(120, 110))
  for j in range(len(i)):
    toRet[int(i[j]) - 1, int(floor(t[j]/0.0011))] = 1
  return toRet

train_data = []
for i in range(len(times)):
  toAppend = convitToMat(spike_data[i], times[i])
  train_data.append(toAppend)
train_data = np.array(train_data)

from sklearn.decomposition import PCA
k = 2 # target dimension(s)
pca_data = []
for i in range(len(train_data)): 
  pca = PCA(5) # Create a new PCA instance
  data = train_data[i].T # 2x2 data matrix
  a = pca.fit_transform(data) # fit and transform
  a = np.array(a)
  a = a.transpose()
  pca_data.append(a)

pca_data = np.array(pca_data)

spike_labels = np.zeros(1000)
spike_labels[0:100] = 0
spike_labels[100:200] = 10
spike_labels[200:300] = 20
spike_labels[300:400] = 30
spike_labels[400:500] = 40
spike_labels[500:600] = 50
spike_labels[600:700] = 60
spike_labels[700:800] = 70
spike_labels[800:900] = 80
spike_labels[900:1000] = 90
spike_labels

flat_pca_data = []
for i in range(len(pca_data)):
  flat_dt = pca_data[i]
  flat_dt = flat_dt.flatten()
  flat_pca_data.append(flat_dt)

In [ ]:
shape(spike_labels)

(1000,)

In [ ]:
flat_pca_data = np.array(flat_pca_data)
X = flat_pca_data
y = spike_labels

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
trainclassifier = KNeighborsClassifier(n_neighbors=10)
trainclassifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [ ]:
y_pred = trainclassifier.predict(X_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred)

0.8133333333333334


array([[35,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 20,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 22, 15,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3, 26,  5,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0, 22,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10, 23,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1, 29,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 23,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  2, 16, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  7, 28]])

In [ ]:
spike_data = []
spike_labels = []
times= []
from matplotlib.image import imread
for files in filenames:
  for k in range(100):
    img = (1-imread(filepath + '/' + files))[::-1, :, 0].T
    num_samples, N = img.shape
    img = gaussian_noise(img)
    start_scope()
    ta = TimedArray(img, dt=1*ms) # 228
    
    number = N #number of neurons
    #a = 0.02/ms
    b = 0.2/ms
    c = -65 * mV
    #d = 6*mV/ms
    
    eqs = '''
    dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u+130*I*1/ms : volt
    I = ta(t, i)*mV                                  : volt
    du/dt = a*(b*v-u)                                : volt/second
    a                                                : 1/second
    d                                                : volt/second
    '''


    reset = '''
    v = c
    u = u + d
    '''

    #Set up neuron population
    G = NeuronGroup(number,eqs,threshold='v >= 30*mV',reset=reset,method='euler')
    G.a = 0.02/ms
    G.d = 8*mV/ms
    G.v = c
    G.u = b * c

    M = SpikeMonitor(G)
    run(num_samples*ms)
    times.append(np.array(M.t))
    spike_data.append(np.array(M.i))
    spike_labels.append(file[:len(file) - 4])
    print("Angle: " + files + ". Repeat: " + str(k))

WARNING    "a" is an internal variable of group "neurongroup_1", but also exists in the run namespace with the value array([[-1.44142217e+00, -1.44142217e+00, -1.44142217e+00, ...,
        -1.44142217e+00, -1.44142217e+00, -1.44142217e+00],
       [-1.87164647e-02, -1.87164647e-02, -1.87164647e-02, ...,
        -1.87164647e-02, -1.87164647e-02, -1.87164647e-02],
       [ 2.15085271e-02,  2.15085271e-02,  2.15085271e-02, ...,
         2.15085271e-02,  2.15085271e-02,  2.15085271e-02],
       [-9.38440504e-03, -9.38440504e-03, -9.38440504e-03, ...,
        -9.38440504e-03, -9.38440504e-03, -9.38440504e-03],
       [ 1.35638788e-04,  1.35638788e-04,  1.35638788e-04, ...,
         1.35638788e-04,  1.35638788e-04,  1.35638788e-04]]). The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    "i" is an internal variable of group "neurongroup_1", but also exists in the run namespace with the value 999. The internal variable will be used. [brian2.gro

Angle: 0.png. Repeat: 0
Angle: 0.png. Repeat: 1
Angle: 0.png. Repeat: 2
Angle: 0.png. Repeat: 3
Angle: 0.png. Repeat: 4
Angle: 0.png. Repeat: 5
Angle: 0.png. Repeat: 6
Angle: 0.png. Repeat: 7
Angle: 0.png. Repeat: 8
Angle: 0.png. Repeat: 9
Angle: 0.png. Repeat: 10
Angle: 0.png. Repeat: 11
Angle: 0.png. Repeat: 12
Angle: 0.png. Repeat: 13
Angle: 0.png. Repeat: 14
Angle: 0.png. Repeat: 15
Angle: 0.png. Repeat: 16
Angle: 0.png. Repeat: 17
Angle: 0.png. Repeat: 18
Angle: 0.png. Repeat: 19
Angle: 0.png. Repeat: 20
Angle: 0.png. Repeat: 21
Angle: 0.png. Repeat: 22
Angle: 0.png. Repeat: 23
Angle: 0.png. Repeat: 24
Angle: 0.png. Repeat: 25
Angle: 0.png. Repeat: 26
Angle: 0.png. Repeat: 27
Angle: 0.png. Repeat: 28
Angle: 0.png. Repeat: 29
Angle: 0.png. Repeat: 30
Angle: 0.png. Repeat: 31
Angle: 0.png. Repeat: 32
Angle: 0.png. Repeat: 33
Angle: 0.png. Repeat: 34
Angle: 0.png. Repeat: 35
Angle: 0.png. Repeat: 36
Angle: 0.png. Repeat: 37
Angle: 0.png. Repeat: 38
Angle: 0.png. Repeat: 39
Angle: 0.p

In [ ]:
#Convert i and t to matrix
def convitToMat(i, t):
  toRet = np.zeros(shape=(120, 110))
  for j in range(len(i)):
    toRet[int(i[j]) - 1, int(floor(t[j]/0.0011))] = 1
  return toRet

train_data = []
for i in range(len(times)):
  toAppend = convitToMat(spike_data[i], times[i])
  train_data.append(toAppend)
train_data = np.array(train_data)

from sklearn.decomposition import PCA
k = 2 # target dimension(s)
pca_data = []
for i in range(len(train_data)): 
  pca = PCA(5) # Create a new PCA instance
  data = train_data[i].T # 2x2 data matrix
  a = pca.fit_transform(data) # fit and transform
  a = np.array(a)
  a = a.transpose()
  pca_data.append(a)

pca_data = np.array(pca_data)

spike_labels = np.zeros(1000)
spike_labels[0:100] = 0
spike_labels[100:200] = 10
spike_labels[200:300] = 20
spike_labels[300:400] = 30
spike_labels[400:500] = 40
spike_labels[500:600] = 50
spike_labels[600:700] = 60
spike_labels[700:800] = 70
spike_labels[800:900] = 80
spike_labels[900:1000] = 90
spike_labels

flat_pca_data = []
for i in range(len(pca_data)):
  flat_dt = pca_data[i]
  flat_dt = flat_dt.flatten()
  flat_pca_data.append(flat_dt)



In [ ]:
flat_pca_data = np.array(flat_pca_data)
X = flat_pca_data
y = spike_labels

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
trainclassifier = KNeighborsClassifier(n_neighbors=6)
trainclassifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [ ]:
y_pred = trainclassifier.predict(X_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred)

0.8966666666666666


array([[24, 11,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 9, 11,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  4, 31,  2,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 32,  2,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 24,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  3, 30,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 30,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 23,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 29,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 35]])

In [ ]:
#Slowly Adapting Dynamic
spike_data = []
spike_labels = []
times= []
from matplotlib.image import imread
for files in filenames:
  for k in range(100):
    img = (1-imread(filepath + '/' + files))[::-1, :, 0].T
    num_samples, N = img.shape
    img = gaussian_noise(img)
    start_scope()
    ta = TimedArray(img, dt=1*ms) # 228

    diffimg = np.zeros_like(img)
    for i in range(len(img)):
      if (i != 0):
        newCol = img[:, i] - img[:, i - 1]
        diffimg[:, i] = newCol
      else:
        newCol = img[:, i]
        diffimg[:,i] = newCol
    dta = TimedArray(diffimg, dt=1*ms)
    
    number = N #number of neurons
    #a = 0.02/ms
    b = 0.2/ms
    c = -65 * mV
    #d = 6*mV/ms
    tdsa = 30*ms
    trsa = 5*ms
    k1 = 0.05 * 1000
    k2 = 3
    k3 = 2
    
    eqs = '''
    dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u+100*Isa*1/ms : volt
    Iin = ta(t, i)*mV                                  : volt
    dIin = dta(t, i)*mV                                : volt
    dIsa/dt = (x - Isa)/tdsa                           :volt
    dx/dt = (k2 * dIin + k1 * Iin - x)/trsa            :volt
    du/dt = a*(b*v-u)                                : volt/second
    a                                                : 1/second
    d                                                : volt/second
    '''


    reset = '''
    v = c
    u = u + d
    '''

    #Set up neuron population
    G = NeuronGroup(number,eqs,threshold='v >= 30*mV',reset=reset,method='euler')
    G.a = 0.02/ms
    G.d = 8*mV/ms
    G.v = c
    G.u = b * c
    G.Isa = 0
    G.x = 0

    M = SpikeMonitor(G)
    run(num_samples*ms)
    times.append(np.array(M.t))
    spike_data.append(np.array(M.i))
    spike_labels.append(file[:len(file) - 4])
    print("Angle: " + files + ". Repeat: " + str(k))

WARNING    "a" is an internal variable of group "neurongroup_1", but also exists in the run namespace with the value array([[-0.41334919, -0.41334919, -0.41334919, ..., -0.54854856,
        -0.38267375, -0.44445016],
       [ 0.0229265 ,  0.0229265 ,  0.0229265 , ..., -0.08981893,
         0.05944458,  0.06879565],
       [-0.09898457, -0.09898457, -0.09898457, ...,  0.14350651,
        -0.10125697, -0.12115118],
       [ 0.02311697,  0.02311697,  0.02311697, ...,  0.63885607,
         0.02489704,  0.02619491],
       [-0.05327011, -0.05327011, -0.05327011, ...,  0.49875223,
        -0.09589034, -0.10159357]]). The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    "a" is an internal variable of group "neurongroup", but also exists in the run namespace with the value array([[-0.41334919, -0.41334919, -0.41334919, ..., -0.54854856,
        -0.38267375, -0.44445016],
       [ 0.0229265 ,  0.0229265 ,  0.0229265 , ..., -0.08981893,
         

Angle: 0.png. Repeat: 0
Angle: 0.png. Repeat: 1
Angle: 0.png. Repeat: 2
Angle: 0.png. Repeat: 3
Angle: 0.png. Repeat: 4
Angle: 0.png. Repeat: 5
Angle: 0.png. Repeat: 6
Angle: 0.png. Repeat: 7
Angle: 0.png. Repeat: 8
Angle: 0.png. Repeat: 9
Angle: 0.png. Repeat: 10
Angle: 0.png. Repeat: 11
Angle: 0.png. Repeat: 12
Angle: 0.png. Repeat: 13
Angle: 0.png. Repeat: 14
Angle: 0.png. Repeat: 15
Angle: 0.png. Repeat: 16
Angle: 0.png. Repeat: 17
Angle: 0.png. Repeat: 18
Angle: 0.png. Repeat: 19
Angle: 0.png. Repeat: 20
Angle: 0.png. Repeat: 21
Angle: 0.png. Repeat: 22
Angle: 0.png. Repeat: 23
Angle: 0.png. Repeat: 24
Angle: 0.png. Repeat: 25
Angle: 0.png. Repeat: 26
Angle: 0.png. Repeat: 27
Angle: 0.png. Repeat: 28
Angle: 0.png. Repeat: 29
Angle: 0.png. Repeat: 30
Angle: 0.png. Repeat: 31
Angle: 0.png. Repeat: 32
Angle: 0.png. Repeat: 33
Angle: 0.png. Repeat: 34
Angle: 0.png. Repeat: 35
Angle: 0.png. Repeat: 36
Angle: 0.png. Repeat: 37
Angle: 0.png. Repeat: 38
Angle: 0.png. Repeat: 39
Angle: 0.p

In [ ]:
#Convert i and t to matrix
def convitToMat(i, t):
  toRet = np.zeros(shape=(120, 110))
  for j in range(len(i)):
    toRet[int(i[j]) - 1, int(floor(t[j]/0.0011))] = 1
  return toRet

train_data = []
for i in range(len(times)):
  toAppend = convitToMat(spike_data[i], times[i])
  train_data.append(toAppend)
train_data = np.array(train_data)

from sklearn.decomposition import PCA
k = 2 # target dimension(s)
pca_data = []
for i in range(len(train_data)): 
  pca = PCA(5) # Create a new PCA instance
  data = train_data[i].T # 2x2 data matrix
  a = pca.fit_transform(data) # fit and transform
  a = np.array(a)
  a = a.transpose()
  pca_data.append(a)

pca_data = np.array(pca_data)

def createSpikeLabels(num):
  spike_labels = np.zeros(num)
  spike_labels[int(0):int(1*num/10)] = 0
  spike_labels[int(1*num/10):int(2*num/10)] = 10
  spike_labels[int(2*num/10):int(3*num/10)] = 20
  spike_labels[int(3*num/10):int(4*num/10)] = 30
  spike_labels[int(4*num/10):int(5*num/10)] = 40
  spike_labels[int(5*num/10):int(6*num/10)] = 50
  spike_labels[int(6*num/10):int(7*num/10)] = 60
  spike_labels[int(7*num/10):int(8*num/10)] = 70
  spike_labels[int(8*num/10):int(9*num/10)] = 80
  spike_labels[int(9*num/10):int(10*num/10)] = 90
  return spike_labels

spike_labels = createSpikeLabels(1000)

flat_pca_data = []
for i in range(len(pca_data)):
  flat_dt = pca_data[i]
  flat_dt = flat_dt.flatten()
  flat_pca_data.append(flat_dt)



In [ ]:
flat_pca_data = np.array(flat_pca_data)
X = flat_pca_data
y = spike_labels

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
trainclassifier = KNeighborsClassifier(n_neighbors=6)
trainclassifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [ ]:
y_pred = trainclassifier.predict(X_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred)

0.9633333333333334


array([[35,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 18,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 37,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  4, 29,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1, 22,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2, 31,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 30,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 23,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 29,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 35]])

In [ ]:
#Rapidly adapting
spike_data = []
spike_labels = []
times= []
from matplotlib.image import imread
for files in filenames:
  for k in range(100):
    img = (1-imread(filepath + '/' + files))[::-1, :, 0].T
    num_samples, N = img.shape
    img = gaussian_noise(img)
    start_scope()
    ta = TimedArray(img, dt=1*ms) # 228

    diffimg = np.zeros_like(img)
    for i in range(len(img)):
      if (i != 0):
        newCol = img[:, i] - img[:, i - 1]
        diffimg[:, i] = newCol
      else:
        newCol = img[:, i]
        diffimg[:,i] = newCol
    dta = TimedArray(diffimg, dt=1*ms)
    
    number = N #number of neurons
    #a = 0.02/ms
    b = 0.2/ms
    c = -65 * mV
    #d = 6*mV/ms
    tra = 30*ms
    k1 = 0.05
    k2 = 3
    k3 = 2
    
    eqs = '''
    dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u+100*Ira*1/ms : volt
    Iin = ta(t, i)*mV                                  : volt
    dIin = 1000*dta(t, i)*mV                                : volt
    dIra/dt = (k3 * ((dIin ** 2) ** 0.5) - Ira) / tra : volt
    du/dt = a*(b*v-u)                                : volt/second
    a                                                : 1/second
    d                                                : volt/second
    '''


    reset = '''
    v = c
    u = u + d
    '''

    #Set up neuron population
    G = NeuronGroup(number,eqs,threshold='v >= 30*mV',reset=reset,method='euler')
    G.a = 0.02/ms
    G.d = 8*mV/ms
    G.v = c
    G.u = b * c
    G.Ira = 0

    M = SpikeMonitor(G)
    run(num_samples*ms)
    times.append(np.array(M.t))
    spike_data.append(np.array(M.i))
    spike_labels.append(file[:len(file) - 4])
    print("Angle: " + files + ". Repeat: " + str(k))






WARNING    "a" is an internal variable of group "neurongroup_1", but also exists in the run namespace with the value array([[ 2.62169903e+00, -2.40522846e-02, -2.40522846e-02, ...,
        -2.40522846e-02, -2.40522846e-02, -2.40522846e-02],
       [ 2.50002704e-17,  2.42110889e-17,  2.42110889e-17, ...,
         3.08280717e-17,  3.08280717e-17,  3.08280717e-17],
       [ 1.94731575e-18,  1.17581305e-16,  9.24914613e-18, ...,
        -6.06308557e-17, -6.06308557e-17, -6.06308557e-17],
       [-8.72407686e-19,  2.33276485e-16, -2.62826492e-17, ...,
         2.58987399e-17,  2.58987399e-17,  2.58987399e-17],
       [ 2.48314180e-20, -7.53827591e-18, -9.51523970e-17, ...,
        -5.01914301e-18, -5.01914301e-18, -5.01914301e-18]]). The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    "a" is an internal variable of group "neurongroup", but also exists in the run namespace with the value array([[ 2.62169903e+00, -2.40522846e-02, -2.40522846e

Angle: 0.png. Repeat: 0
Angle: 0.png. Repeat: 1
Angle: 0.png. Repeat: 2
Angle: 0.png. Repeat: 3
Angle: 0.png. Repeat: 4
Angle: 0.png. Repeat: 5
Angle: 0.png. Repeat: 6
Angle: 0.png. Repeat: 7
Angle: 0.png. Repeat: 8
Angle: 0.png. Repeat: 9
Angle: 0.png. Repeat: 10
Angle: 0.png. Repeat: 11
Angle: 0.png. Repeat: 12
Angle: 0.png. Repeat: 13
Angle: 0.png. Repeat: 14
Angle: 0.png. Repeat: 15
Angle: 0.png. Repeat: 16
Angle: 0.png. Repeat: 17
Angle: 0.png. Repeat: 18
Angle: 0.png. Repeat: 19
Angle: 0.png. Repeat: 20
Angle: 0.png. Repeat: 21
Angle: 0.png. Repeat: 22
Angle: 0.png. Repeat: 23
Angle: 0.png. Repeat: 24
Angle: 0.png. Repeat: 25
Angle: 0.png. Repeat: 26
Angle: 0.png. Repeat: 27
Angle: 0.png. Repeat: 28
Angle: 0.png. Repeat: 29
Angle: 0.png. Repeat: 30
Angle: 0.png. Repeat: 31
Angle: 0.png. Repeat: 32
Angle: 0.png. Repeat: 33
Angle: 0.png. Repeat: 34
Angle: 0.png. Repeat: 35
Angle: 0.png. Repeat: 36
Angle: 0.png. Repeat: 37
Angle: 0.png. Repeat: 38
Angle: 0.png. Repeat: 39
Angle: 0.p

In [ ]:
#Convert i and t to matrix
def convitToMat(i, t):
  toRet = np.zeros(shape=(120, 110))
  for j in range(len(i)):
    toRet[int(i[j]) - 1, int(floor(t[j]/0.0011))] = 1
  return toRet

train_data = []
for i in range(len(times)):
  toAppend = convitToMat(spike_data[i], times[i])
  train_data.append(toAppend)
train_data = np.array(train_data)

from sklearn.decomposition import PCA
k = 2 # target dimension(s)
pca_data = []
for i in range(len(train_data)): 
  pca = PCA(5) # Create a new PCA instance
  data = train_data[i].T # 2x2 data matrix
  a = pca.fit_transform(data) # fit and transform
  a = np.array(a)
  a = a.transpose()
  pca_data.append(a)

pca_data = np.array(pca_data)

def createSpikeLabels(num):
  spike_labels = np.zeros(num)
  spike_labels[int(0):int(1*num/10)] = 0
  spike_labels[int(1*num/10):int(2*num/10)] = 10
  spike_labels[int(2*num/10):int(3*num/10)] = 20
  spike_labels[int(3*num/10):int(4*num/10)] = 30
  spike_labels[int(4*num/10):int(5*num/10)] = 40
  spike_labels[int(5*num/10):int(6*num/10)] = 50
  spike_labels[int(6*num/10):int(7*num/10)] = 60
  spike_labels[int(7*num/10):int(8*num/10)] = 70
  spike_labels[int(8*num/10):int(9*num/10)] = 80
  spike_labels[int(9*num/10):int(10*num/10)] = 90
  return spike_labels

spike_labels = createSpikeLabels(1000)

flat_pca_data = []
for i in range(len(pca_data)):
  flat_dt = pca_data[i]
  flat_dt = flat_dt.flatten()
  flat_pca_data.append(flat_dt)



WARNING    /usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
 [py.warnings]
  explained_variance_ratio_ = explained_variance_ / total_var

WARNING    /usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
 [py.warnings]
  explained_variance_ratio_ = explained_variance_ / total_var

WARNING    /usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
 [py.warnings]
  explained_variance_ratio_ = explained_variance_ / total_var

WARNING    /usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_

In [ ]:
flat_pca_data = np.array(flat_pca_data)
X = flat_pca_data
y = spike_labels

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
trainclassifier = KNeighborsClassifier(n_neighbors=6)
trainclassifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [ ]:
y_pred = trainclassifier.predict(X_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred)

0.09333333333333334


array([[ 4,  8,  3,  2,  2,  5,  4,  3,  4,  0],
       [ 3,  4,  2,  1,  2,  1,  4,  0,  3,  0],
       [ 5,  8,  4,  2,  0,  6,  5,  2,  5,  0],
       [ 2,  5,  3,  1,  1,  0, 15,  2,  5,  0],
       [ 3,  3,  3,  0,  1,  7,  6,  0,  1,  0],
       [ 8,  7,  4,  1,  2,  3,  2,  2,  3,  1],
       [ 4,  4,  1,  2,  0,  1,  5,  5,  7,  1],
       [ 4,  9,  1,  1,  1,  2,  4,  0,  1,  0],
       [ 1,  4,  2,  1,  4,  5,  3,  2,  6,  1],
       [ 5,  7,  2,  1,  3,  3, 10,  1,  3,  0]])